# Bogumił Kamiński, 2019-03-25

In [1]:
using LinearAlgebra
using Random
using ForwardDiff
using DataFrames
using GLM

In [2]:
function marquardt(f, x₀; ε=1e-6, maxiter=1000, λ=10.0^4, α=2)
    x = x₀
    fx = f(x)
    converged = false
    i = 0
    while i < maxiter
        i += 1
        ∇f = ForwardDiff.gradient(f, x)
        if norm(∇f) ≤ ε
            converged = true
            break
        end
        ∇²f = ForwardDiff.hessian(f, x)
        x′ = x - (∇²f + λ*I) \ ∇f
        fx′ = f(x′)
        if fx′ < fx
            λ *= 0.5
            fx = fx′
            x = x′
        else
            λ *= 2.0
        end
    end
    (x=x, fx=fx, converged=converged, iters=i)
end

marquardt (generic function with 1 method)

In [3]:
rosenbrock(x) = sum((1-x[i])^2 + 100(x[i+1]-x[i]^2)^2 for i in 1:length(x)-1)

rosenbrock (generic function with 1 method)

In [4]:
Random.seed!(1234)
x = rand(20);
marquardt(rosenbrock, x)

(x = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], fx = 7.643342734267342e-15, converged = true, iters = 53)

In [5]:
Random.seed!(123)
N = 1000
df = DataFrame(rand(N, 4))
df.y = sum.(eachrow(df)) + randn(N);
df

,x1,x2,x3,x4,y
,Float64,Float64,Float64,Float64,Float64
1,0.768448,0.724897,0.0824815,0.240819,1.76327
2,0.940515,0.0745399,0.950025,0.396575,3.32997
3,0.673959,0.317564,0.657439,0.878578,3.46812
4,0.395453,0.286774,0.147889,0.855912,1.59305
5,0.313244,0.702091,0.191318,0.572289,1.99789
6,0.662555,0.497805,0.318801,0.955608,3.04278
7,0.586022,0.0628874,0.0241049,0.582165,0.946514
8,0.0521332,0.817261,0.170354,0.309421,2.0729
9,0.26864,0.412464,0.679465,0.324786,1.79548


In [6]:
lm(@formula(y~x1+x2+x3+x4), df)

StatsModels.DataFrameRegressionModel{LinearModel{LmResp{Array{Float64,1}},DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: y ~ 1 + x1 + x2 + x3 + x4

Coefficients:
             Estimate Std.Error t value Pr(>|t|)
(Intercept)  0.306163  0.118228  2.5896   0.0097
x1           0.585532  0.117601 4.97897    <1e-6
x2           0.928859  0.114768 8.09336   <1e-14
x3            0.88908  0.115616 7.68994   <1e-13
x4           0.976342  0.113607 8.59403   <1e-16


In [7]:
function ssq(x)
    m = Matrix(df)[:, startswith.(string.(names(df)), "x")]
    norm(df.y - fill(x[1], nrow(df)) - m * x[2:end])
end
marquardt(ssq, rand(5))

(x = [0.306163, 0.585532, 0.928859, 0.88908, 0.976342], fx = 32.91927364835108, converged = true, iters = 20)